# Mode-1

In [ ]:
import gmsh
import math
import sys

gmsh.initialize()
gmsh.model.add("mode1_crack")

# ----------------------------
# Parameters
# ----------------------------
L = 1.0       # Length of square
H = 1.0       # Height of square
cmod = 0.001   # Crack mouth opening displacement (total)
lc = 0.4     # Characteristic mesh length

# Half opening on top and bottom faces at x = 0
delta = cmod / 2.0

# ----------------------------
# Geometry definition
# ----------------------------

# Points on bottom edge
p1 = gmsh.model.geo.addPoint(0, 0, 0, lc)
p2 = gmsh.model.geo.addPoint(0.5, 0, 0, lc)
p3 = gmsh.model.geo.addPoint(1, 0, 0, lc)

# Crack points (at mid-height)
p4 = gmsh.model.geo.addPoint(0.0, 0.5 - delta, 0, lc)
p5 = gmsh.model.geo.addPoint(0.0, 0.5 + delta, 0, lc)
p6 = gmsh.model.geo.addPoint(0.5, 0.5, 0, lc)
p7 = gmsh.model.geo.addPoint(1.0, 0.5, 0, lc)

# Points on top edge
p8 = gmsh.model.geo.addPoint(0, H, 0, lc)
p9 = gmsh.model.geo.addPoint(0.5, H, 0, lc)
p10 = gmsh.model.geo.addPoint(1, H, 0, lc)

# ----------------------------
# Lines and crack segments
# ----------------------------

# Bottom half (below crack)
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p7)
l4 = gmsh.model.geo.addLine(p7, p6) 
l5 = gmsh.model.geo.addLine(p6, p4)
l6 = gmsh.model.geo.addLine(p4, p1)


# Top half (above crack)
l7 = gmsh.model.geo.addLine(p6, p5)
l9 = gmsh.model.geo.addLine(p5, p8)
l10 = gmsh.model.geo.addLine(p8, p9)
l11 = gmsh.model.geo.addLine(p9, p10)
l12 = gmsh.model.geo.addLine(p10, p7)


# ----------------------------
# Surfaces
# ----------------------------
loop1 = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4, l5, l6])
surf1 = gmsh.model.geo.addPlaneSurface([loop1])

loop2 = gmsh.model.geo.addCurveLoop([l4, l7, l9, l10, l11, l12])
surf2 = gmsh.model.geo.addPlaneSurface([loop2])

gmsh.model.geo.synchronize()

# ----------------------------
# Physical groups
# ----------------------------
gmsh.model.addPhysicalGroup(2, [surf1, surf2], name="Domain")
gmsh.model.addPhysicalGroup(1, [l1, l2, l3, l5, l6, l7, l9, l10, l11, l12], name="ExternalBoundary")

# ----------------------------
# Mesh generation
# ----------------------------
gmsh.model.mesh.generate(2)

# Save to file
gmsh.write("mode1_crack.msh")
gmsh.finalize()


In [ ]:
# ------------------------------------------------------------------
# Convert .msh → .xdmf for FEniCS
# ------------------------------------------------------------------
import meshio
for name in ["mode1_crack"]:
    msh = meshio.read(f"{name}.msh")
    # FEniCS expects only cells and points
    tri_cells = {"triangle": msh.cells["triangle"]}
    mesh = meshio.Mesh(points=msh.points[:, :2], cells=tri_cells)
    meshio.write(f"{name}.xdmf", mesh)
    print(f"✅ Saved {name}.xdmf")



In [ ]:
meshio.write("mode1_line.xdmf", meshio.Mesh(
    points=msh.points[:, :2],
    cells={"line": msh.cells["line"]},
    cell_data={"line": {"marker": msh.cell_data["line"]['gmsh:physical']}}
))

In [ ]:
msh.field_data